In [30]:

!pip install -q transformers torch sentencepiece PyMuPDF pdfminer.six nltk gradio

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [31]:

import torch, sys
print("Python:", sys.version.splitlines()[0])
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        print("GPU name:", torch.cuda.get_device_name(0))
    except Exception as e:
        print("Couldn't fetch GPU name:", e)


Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Torch version: 2.6.0+cu124
CUDA available: True
GPU name: Tesla T4


In [32]:
from google.colab import files
uploaded = files.upload()
print("Uploaded:", list(uploaded.keys()))


Saving 1686821361_RBI_-_DLG_Guidelines_(June_8,_2023).pdf to 1686821361_RBI_-_DLG_Guidelines_(June_8,_2023) (4).pdf
Uploaded: ['1686821361_RBI_-_DLG_Guidelines_(June_8,_2023) (4).pdf']


In [33]:
import fitz


pdf_path = "/content/1686821361_RBI_-_DLG_Guidelines_(June_8,_2023).pdf"


doc = fitz.open(pdf_path)
full_text = ""

for page in doc:
    text = page.get_text()
    full_text += text + "\n"

print("Total characters extracted:", len(full_text))
print("\n--- Preview of first 1000 characters ---\n")
print(full_text[:1000])


Total characters extracted: 9832

--- Preview of first 1000 characters ---

 
www.rbi.org.in  
 
RBI/2023-24/41 
DOR.CRE.REC.21/21.07.001/2023-24  
 
 
 
       June 08, 2023 
 
All Commercial Banks (including Small Finance Banks), 
Primary (Urban) Co-operative Banks, State Co-operative Banks, 
Central Co-operative Banks; and 
Non-Banking Financial Companies (including Housing Finance Companies) 
Madam/ Dear Sir, 
Guidelines on Default Loss Guarantee (DLG) in Digital Lending 
A reference is invited to Para (3.4.3.1) of Section C of Annex-II to the RBI Press Release 
“Recommendations of the Working group on Digital Lending – Implementation” dated 
August 10, 2022 in terms of which it was stated that the recommendation pertaining to 
First Loss Default Guarantee (FLDG) was under examination with the Reserve Bank. 
2. Arrangements between Regulated Entities (REs) and Lending Service Providers 
(LSPs) or between two REs involving default loss guarantee (DLG), commonly known as 
FLDG, has s

In [34]:
import re

clean_text = full_text

clean_text = re.sub(r'\n+', ' ', clean_text)
clean_text = re.sub(r'\s+', ' ', clean_text)


clean_text = re.sub(r'http\S+|www\.\S+', '', clean_text)


clean_text = re.sub(r'RBI/\d{4}-\d{2}/\d+', '', clean_text)


clean_text = clean_text.strip()

print("Characters after cleaning:", len(clean_text))
print("\n--- Preview of cleaned text ---\n")
print(clean_text[:1000])


Characters after cleaning: 9364

--- Preview of cleaned text ---

DOR.CRE.REC.21/21.07.001/2023-24 June 08, 2023 All Commercial Banks (including Small Finance Banks), Primary (Urban) Co-operative Banks, State Co-operative Banks, Central Co-operative Banks; and Non-Banking Financial Companies (including Housing Finance Companies) Madam/ Dear Sir, Guidelines on Default Loss Guarantee (DLG) in Digital Lending A reference is invited to Para (3.4.3.1) of Section C of Annex-II to the RBI Press Release “Recommendations of the Working group on Digital Lending – Implementation” dated August 10, 2022 in terms of which it was stated that the recommendation pertaining to First Loss Default Guarantee (FLDG) was under examination with the Reserve Bank. 2. Arrangements between Regulated Entities (REs) and Lending Service Providers (LSPs) or between two REs involving default loss guarantee (DLG), commonly known as FLDG, has since been examined by the Bank and it has been decided to permit such arrange

In [36]:
!pip install transformers sentencepiece --quiet

from transformers import pipeline


summarizer = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6",
    device=-1
)


def chunk_text(text, max_words=400):
    words = text.split()
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

chunks = chunk_text(clean_text)

summaries = []
for i, chunk in enumerate(chunks):
    print(f"\n--- Summarizing chunk {i+1}/{len(chunks)} ---")
    summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
    summaries.append(summary)

final_summary = " ".join(summaries)

print("\n===== FINAL SUMMARY =====\n")
print(final_summary)


Device set to use cpu



--- Summarizing chunk 1/4 ---

--- Summarizing chunk 2/4 ---

--- Summarizing chunk 3/4 ---

--- Summarizing chunk 4/4 ---

===== FINAL SUMMARY =====

 First Loss Default Guarantee (FLDG) was under examination with the Reserve Bank . Arrangements between Regulated Entities (REs) and Lending Service Providers (LSPs) involving default loss guarantee (DLG) has since been examined by the Bank and it has been decided to permit such arrangements subject to guidelines laid down in the Annex to this circular .  These guidelines are applicable to DLG arrangements entered in ‘Digital Lending1’ operations undertaken by following entities (hereinafter referred to as ‘Regulated Entities’) These guidelines will have the same meaning as assigned to them in the Circular on ‘Guidelines on Digital Lending’ dated September 02, 2022 .  The amount of DLG invoked shall not be set off against the underlying individual loans . Recovery by the RE, if any, from the loans on which DLG has been invoked and reali

In [37]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [38]:


!pip install -q scikit-learn

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')


from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(clean_text)


vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9)
tfidf = vectorizer.fit_transform(sentences)

import numpy as np
scores = tfidf.sum(axis=1).A1

N = 8
top_idx = np.argsort(scores)[-N:][::-1]
top_sentences = [sentences[i] for i in top_idx]

print("Top sentences (raw):\n")
for s in top_sentences:
    print("-", s)
print("\n")


from transformers import pipeline

summ = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=-1)

combined = " ".join(top_sentences)

polished = summ(combined, max_length=180, min_length=40, do_sample=False)[0]['summary_text']

bullets = [b.strip() for b in polished.split('. ') if b.strip()]

final_bullets = bullets[:6]

print("Final bullets:\n")
for i, b in enumerate(final_bullets, 1):

    if not b.endswith('.'):
        b = b + '.'
    print(f"{i}. {b}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Top sentences (raw):

- DOR.CRE.REC.21/21.07.001/2023-24 June 08, 2023 All Commercial Banks (including Small Finance Banks), Primary (Urban) Co-operative Banks, State Co-operative Banks, Central Co-operative Banks; and Non-Banking Financial Companies (including Housing Finance Companies) Madam/ Dear Sir, Guidelines on Default Loss Guarantee (DLG) in Digital Lending A reference is invited to Para (3.4.3.1) of Section C of Annex-II to the RBI Press Release “Recommendations of the Working group on Digital Lending – Implementation” dated August 10, 2022 in terms of which it was stated that the recommendation pertaining to First Loss Default Guarantee (FLDG) was under examination with the Reserve Bank.
- 4 Para 7 of the ‘Master Circular on Basel III Capital Regulations’ dated May 12, 2023, Master Direction - Non-Banking Financial Company – Non-Systemically Important Non-Deposit taking Company (Reserve Bank) Directions, 2016 (as updated from time to time), Master Direction - Non-Banking Fina

Device set to use cpu


Final bullets:

1. A reference is invited to Para (3.4.3.1) of Section C of Annex-II to the RBI Press Release “Recommendations of the Working group on Digital Lending – Implementation” dated August 10, 2022.
2. It was stated that the recommendation pertaining to First Loss Default Guarantee (FLDG) was under examination with the Reserve Bank .


In [39]:
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=-1
)


Device set to use cpu


In [40]:
from transformers import pipeline
import nltk
import PyPDF2

nltk.download('punkt')
nltk.download('punkt_tab')

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)

pdf_path = "1686821361_RBI_-_DLG_Guidelines_(June_8,_2023).pdf"
pdf_reader = PyPDF2.PdfReader(open(pdf_path, "rb"))
raw_text = ""
for page in pdf_reader.pages:
    raw_text += page.extract_text()

from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(raw_text)

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)
scores = np.asarray(X.sum(axis=1)).ravel()
sentence_scores = {sentences[i]: scores[i] for i in range(len(sentences))}
top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:7]

summary_text = summarizer(raw_text[:3000], max_length=150, min_length=50, do_sample=False)[0]['summary_text']

print("Top Sentences from TF-IDF:")
for sent in top_sentences:
    print("-", sent)

print("\nBART Model Summary:\n", summary_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Device set to use cpu


Top Sentences from TF-IDF:
- 21/21.0 7.001/2023-24            June 08, 2023 
 All Commercial Banks  (including Small Finance Banks),  
Primary (Urban) Co -operative Banks, State Co- operative Banks,  
Central Co -operative Banks; and 
Non-Banking Financial Companies (including Housing Finance Companies)  
Madam/  Dear Sir,  
Guidelines on Default Loss Guarantee (DLG ) in Digital Lending 
A reference is invited to Para (3.4.3.1) of Section C of Annex -II to the RBI Press Release 
“Recommendations of the Working group on Digital Lending – Implementation” dated 
August 10, 2022 in terms of which it was stated that the recommendation pertaining to 
First Loss Default Guarantee (FLDG) was under examination with the Reserve Bank .
- Arrangement s between Regulated Entities ( REs) and Lending Service Providers 
(LSPs ) or between two REs involving default loss guarantee (DLG) , commonly known as 
FLDG , has since been examined by the Bank and it has been decided to permit such 
arrangements s

In [41]:
from transformers import pipeline
import nltk
import fitz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from google.colab import files

nltk.download("punkt")
nltk.download("punkt_tab")

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]
pdf_document = fitz.open(pdf_path)
raw_text = ""
for page in pdf_document:
    raw_text += page.get_text()

sentences = nltk.sent_tokenize(raw_text)
vectorizer = TfidfVectorizer()
sentence_vectors = vectorizer.fit_transform(sentences)
similarity_matrix = cosine_similarity(sentence_vectors)
scores = similarity_matrix.sum(axis=1)
sentence_scores = {sent: score for sent, score in zip(sentences, scores)}
top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:7]

summary_text = summarizer(raw_text[:3000], max_length=150, min_length=50, do_sample=False)[0]['summary_text']
print("Extractive Summary:\n", " ".join(top_sentences))
print("\nAbstractive Summary:\n", summary_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Device set to use cpu


Saving 1686821361_RBI_-_DLG_Guidelines_(June_8,_2023).pdf to 1686821361_RBI_-_DLG_Guidelines_(June_8,_2023) (5).pdf
Extractive Summary:
 Such contract, 
among other things, must contain the following details: 
                                                            
1 As defined under Para 2.3 of the Annex to the Circular on ‘Guidelines on Digital Lending’ dated September 02, 2022 
2 DLG shall not involve any actual transfer of the underlying loan exposure from the books of the RE to the books of the DLG Provider. Any other implicit 
guarantee of similar nature linked to the performance of the loan portfolio of 
the RE and specified upfront, shall also be covered under the definition of DLG. Such policy shall include, at the minimum, the eligibility criteria 
for DLG provider, nature and extent of DLG cover, process of monitoring and 
reviewing the DLG arrangement, and the details of the fees, if any, payable to 
the DLG provider. Recovery by the RE, if any, from the loans on which

In [42]:
!pip install transformers pypdf2 nltk torch

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

from PyPDF2 import PdfReader
from transformers import pipeline
import torch
from nltk.tokenize import sent_tokenize

device = -1

pdf_path = "1686821361_RBI_-_DLG_Guidelines_(June_8,_2023).pdf"
reader = PdfReader(pdf_path)
raw_text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()])

sentences = sent_tokenize(raw_text)
sentence_scores = {sentence: len(sentence.split()) for sentence in sentences}
top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:7]

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
summary_text = summarizer(raw_text[:3000], max_length=150, min_length=50, do_sample=False)[0]['summary_text']

print("\n" + "="*80)
print("# 📄 RBI DLG Guidelines Summary (Clean & Professional)")
print("="*80 + "\n")
print("## 📝 Extracted Key Sentences:\n")
for i, sentence in enumerate(top_sentences, 1):
    print(f"{i}. {sentence}")
print("\n## 📌 Model-Generated Summary:\n")
print(summary_text)
print("\n" + "="*80)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Device set to use cpu



# 📄 RBI DLG Guidelines Summary (Clean & Professional)

## 📝 Extracted Key Sentences:

1. 21/21.0 7.001/2023-24            June 08, 2023 
 All Commercial Banks  (including Small Finance Banks),  
Primary (Urban) Co -operative Banks, State Co- operative Banks,  
Central Co -operative Banks; and 
Non-Banking Financial Companies (including Housing Finance Companies)  
Madam/  Dear Sir,  
Guidelines on Default Loss Guarantee (DLG ) in Digital Lending 
A reference is invited to Para (3.4.3.1) of Section C of Annex -II to the RBI Press Release 
“Recommendations of the Working group on Digital Lending – Implementation” dated 
August 10, 2022 in terms of which it was stated that the recommendation pertaining to 
First Loss Default Guarantee (FLDG) was under examination with the Reserve Bank .
2. 4 Para 7 of the ‘Master Circular on Basel III Capital Regulations’ dated May 12, 2023, Master Direction -  Non- Banking Financial 
Company – Non-Systemically Important Non- Deposit taking Company (Rese